In [1]:
import re
import sys
from time import sleep
import datetime


import numpy as np
from sklearn.externals import joblib
from sklearn.externals.joblib import Parallel, delayed
from tqdm import *
import requests
from user_agent import generate_user_agent
from bs4 import BeautifulSoup

In [2]:
BASE_URL = 'https://www.avito.ru'
MONET_URL = BASE_URL + '/moskva/kollektsionirovanie/monety'

_____

In [3]:
proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]


for i in range(len(proxies_list)):
    sleep(np.random.random())
    assert 200 == requests.get('https://yandex.ru', 
                               headers={'User-Agent': generate_user_agent()},
                               proxies=proxies_list[i]).status_code
print ('all_proxies_are_worked!\n\n')

for i in range(len(proxies_list)):
    assert 200 == requests.get('https://avito.ru', 
                               headers={'User-Agent': generate_user_agent()},
                               proxies=proxies_list[i]).status_code
print ('all_proxies_are_worked_at_avito!\n\n')

def get_proxy():
    return proxies_list[np.random.randint(0, len(proxies_list))]

all_proxies_are_worked!


all_proxies_are_worked_at_avito!




In [4]:
def get_me_date(txt): 
    m = re.findall('\D{1}(\d{3,4})\D{1}', ' '+txt+' ')
    m = np.array(m).astype(int)
    return m[(m < 2100) & (m > 600) & (m != 925)].astype(int)

assert np.all(np.array([1992, 1993, 1994, 889, 1000, 1995]) == \
            get_me_date('1992 ghbdtn 1993 fsuer 215 руб dv u2194r '+\
                        'trgg 1994 и 889-991 года b tckb njkm4rj 1000 '+\
                        '23452 1995'))

_____

In [5]:
dates = {} # dictionary - URL:title_text
all_refs = [] # list of URLs on pages with descriptions

In [6]:
for i in tqdm(range(1,200), desc='parsing_pages:'):
    sleep(np.random.random() + 0.92)
    req = requests.get(MONET_URL, 
                   params={'p':i, 'view':'list', 'pmax':1000},
                   headers={'User-Agent': generate_user_agent()},
                   proxies=get_proxy())
    if MONET_URL == req.url:
        break
    if str(i) not in req.url:
        print ('no_page_with_num_'+str(i)+' - Stoped')
        break
        #raise Exception('no_page_with_num_'+str(i))
    if 200 != req.status_code:
        print (i, req.status_code)
        continue
    soup = BeautifulSoup(req.content, 'html.parser')
    work_item = soup.find_all('a', class_='description-title-link')
    for item in work_item:
        all_refs.append(item.attrs['href'])
        dates[item.attrs['href']] = {'title':item.text}

parsing_pages::  95%|█████████▌| 190/199 [06:01<00:15,  1.69s/it]

no_page_with_num_191 - Stoped


In [7]:
for i in tqdm(range(1,186), desc='parsing_pages:'):
    sleep(np.random.random() + 0.72)
    req = requests.get(MONET_URL, 
                       params={'p':i, 'view':'list', 'pmin':1001}, 
                       headers={'User-Agent': generate_user_agent()},
                       proxies=get_proxy())
    if MONET_URL == req.url:
        break
    if str(i) not in req.url:
        print ('no_page_with_num_'+str(i)+' - Stoped')
        break
        #raise Exception('no_page_with_num_'+str(i))
    if 200 != req.status_code:
        print (req.status_code, i)
        continue
        
    soup = BeautifulSoup(req.content, 'html.parser')
    work_item = soup.find_all('a', class_='description-title-link')
    for item in work_item:
        all_refs.append(item.attrs['href'])
        dates[item.attrs['href']] = {'title':item.text}


parsing_pages::  86%|████████▌ | 159/185 [04:30<00:43,  1.66s/it]

no_page_with_num_160 - Stoped


In [8]:
def get_statistics_title(dates):
    tmp_title_dates = []
    zeros_title = 0
    for url in dates:
        title_dates = get_me_date(dates[url]['title'])
        if len(title_dates) > 0:
            tmp_title_dates.append(title_dates[0])
        else:
            zeros_title += 1
    monets = np.array(tmp_title_dates)
    ratio_before_2000_to_after = np.sum(monets < 2000) / np.sum(monets >= 2000)
    return (zeros_title, ratio_before_2000_to_after)

In [9]:
get_statistics_title(dates)

(15056, 1.5878564857405704)

### Можно еще уточнить по описанию, но мне влом.